In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import fsolve
import casadi as ca
import matplotlib.pyplot as plt


2024-09-23 16:46:13.530997: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 16:46:13.539260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 16:46:13.548838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 16:46:13.551783: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 16:46:13.558841: I tensorflow/core/platform/cpu_feature_guar

In [7]:
np.random.seed(42)
alpha0 = np.random.uniform(0.2, 0.8)
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 8.5
P_out = 5
C = 479

def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)
    eqn_2 = (C**2)/2 * (x - alpha0 * kv * np.sqrt(y - P_out))
    return [eqn_1, eqn_2]


result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

a = result[0]
b = result[1]
interval = [np.linspace(0,400,400), np.linspace(400,800,400),np.linspace(800,1200,400),np.linspace(1200,1600,400),np.linspace(1600,2000,400)]
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)
x0_values = []
x1_values = []
alpha_values = [np.full(400, alpha0)]


for i in range(0,5):  
    if i ==0:
        alpha1 = alpha0
    else:
        alpha1 = np.random.uniform(0.2, 0.8)
        alpha_values.append(np.full(400, alpha1))
    
    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1]), (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1] - P_out)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[i][0], interval[i])
    
    sol = F(x0 = [a, b], p = alpha1)

    xf_values = np.array(sol["xf"])

    aux1, aux2 = xf_values
    x0_values.append(aux1)
    x1_values.append(aux2)
    a = aux1[-1]
    b = aux2[-1]

'''
plt.figure("mass flow rate x time")
for i in range(0,5):
    plt.plot(interval[i], np.squeeze(x0_values[i]), label='x0(t)')
plt.grid(True)
plt.xlabel('Time')  
plt.ylabel('Value')

plt.figure("Plenum pressure x time")
for i in range(0,5):
    plt.plot(interval[i], np.squeeze(x1_values[i]), label='x0(t)')
plt.grid(True)
plt.xlabel('Time')  
plt.ylabel('Value')

plt.figure("alpha x time")
for i in range(0,5):
    plt.plot(interval[i], np.squeeze(alpha_values[i]), linestyle=':')
plt.grid(True)
plt.xlabel('time') 
plt.ylabel('Value')


plt.show()
'''
data = []
data0 = np.reshape(x0_values, -1)
data1 = np.reshape(x1_values, -1)
data2 = np.reshape(alpha_values, -1)


In [19]:
# Gerar os dados
for i in range (2000):
    data.append([data0[i], data1[i], data2[i]])
print(data)
X = []
y = []

# Criar sequências de entrada e saída
for i in range(len(data) - 1):
    X.append(data[i]) # entrada: o número atual
    y.append(data[i + 1]) # saída: o próximo número

X = np.array(X)
y = np.array(y)

X = X.reshape((X.shape[0], 1, X.shape[1]))
print(X)

model = keras.Sequential([
    layers.LSTM(50, activation='tanh', return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    layers.LSTM(50, activation = 'tanh'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=10, verbose=0)

test_input = np.array([[0.5924603766348946, 12.773645605119716, 0.559195090518222], []]).reshape((1,1,3))

predicted = model.predict(test_input, verbose=0)

print(f"Próximo número na sequência: {predicted}")


[[0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746, 12.75, 0.42472407130841755], [0.449305074170746,